In [7]:
from dotenv import load_dotenv

load_dotenv()
import nest_asyncio
import os

nest_asyncio.apply()
import json
import base64

In [8]:
# Load json from data/data.json

with open("data/data.json") as f:
    data = json.load(f)
print(data["script"])

# Constants:
img_url = """https://cdn.discordapp.com/attachments/1212193794494042202/1223728760948523190/jas1.jpg?ex=661ae938&is=66087438&hm=a2661c174ebec9afba3265e3eb6bb86832f732f66d88195a5faa293a09839654&"""
voice_id = "jA3XNbtepbUtYOz5Q5bI"
bearer = os.environ.get("DID_API_KEY")
encoded = base64.b64encode(bearer.encode("utf-8")).decode("utf-8")
eleven = os.environ.get("ELEVENLABS_API_KEY")

Imagine you're at a popular salad place and you see a huge shelf with 26 sections labeled A to Z. That's a real-life example of a data structure called a hash table! 🥗📊

Here's how it works: when you order a salad, they put it on the shelf based on the first letter of your name. So if your name is Alice, your salad goes in the 'A' section. If it's Bob, it goes in the 'B' section. And so on!

This lets them find your salad super fast when you come to pick it up. Instead of searching through the whole shelf, they just go straight to your letter. Computer scientists call this 'constant time' or 'O(1)' ⏱️

But there's a catch! If tons of people with the same first letter order salads at the same time, that section can get overcrowded and they might have to start putting salads in the wrong place. Then it becomes a mess! 😅

So next time you see something organized from A-Z, you'll know there's some clever computer science going on behind the scenes! 🤓


In [9]:
import asyncio
import aiohttp

async def post_talk(script):
    url = "https://api.d-id.com/talks"
    headers = {
        'accept': 'application/json',
        'authorization': f'Basic {encoded}',
        'Content-Type': 'application/json',
        'x-api-key-external': json.dumps({"elevenlabs": eleven})
    }
    data = {
        "script": {
            "type": "text",
            "subtitles": "false",
            "provider": {
                "type": "elevenlabs",
                "voice_id": voice_id,
                "voice_config": {"stability": 0.3, "similarity_boost": 0.9},
            },
            "model_id": "eleven_multilingual_v2",
            "input": script,
        },
        "config": {"fluent": "false", "pad_audio": "0.0"},
        "source_url": img_url,
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=data) as response:
            # Check if the request was successful
            if response.status == 201:
                # Process the response here
                response_data = await response.json()
                id = response_data["id"]

            else:
                # Handle request errors here
                print(f"Failed to post data, status code: {response.status}")
                print(await response.text())
                return ""

        # Loop to check the status
        while True:
            async with session.get(f"{url}/{id}", headers=headers) as status_response:
                if status_response.status == 200:
                    status_data = await status_response.json()
                    if status_data["status"] == "done":
                        result_url = status_data["result_url"]
                        break
                    else:
                        # Wait for some time before checking the status again
                        await asyncio.sleep(5)
                else:
                    print(f"Failed to get data, status code: {status_response.status}")
                    return ""

        # Download the video
        async with session.get(result_url) as video_response:
            if video_response.status == 200:
                mp4_data = (
                    await video_response.read()
                )  # This is the binary data of the MP4 file
                return mp4_data
            else:
                print(f"Failed to download video, status code: {video_response.status}")
                return ""

In [10]:
video = asyncio.run(post_talk(data["script"]))

In [ ]:
# Save video
with open("data/headshot.mp4", "wb") as f:
    f.write(video)